In [1]:
from pydantic import BaseModel, field_validator, ValidationError
from typing import Any
import pytz
from dateutil.parser import parse
from datetime import datetime

In [2]:
class Model(BaseModel):
    dt: datetime

    @field_validator("dt", mode="before")
    @classmethod
    def parse_datetime(cls, value: Any):
        if isinstance(value, str):
            try:
                return parse(value)
            except Exception as ex:
                raise ValidationError(str(ex))
        return value

    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        
        return dt
            



In [3]:
Model(dt=100000)

Model(dt=datetime.datetime(1970, 1, 2, 3, 46, 40, tzinfo=<UTC>))

In [4]:
eastern = pytz.timezone("US/Eastern")
dt = eastern.localize(datetime(2020, 1, 1, 3, 0, 0))
dt

datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [5]:
Model(dt=dt)

Model(dt=datetime.datetime(2020, 1, 1, 8, 0, tzinfo=<UTC>))

In [6]:
# Before validators execute bottom up
# After validators execute top to bottom